# Exploratory Data Analysis - E-commerce Dataset

This notebook performs exploratory data analysis on the e-commerce data to:
1. Join CSV files together to develop a comprehensive dataset
2. Answer which categories have the highest revenue
3. Answer how sales change over time

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## 2. Load CSV Files

In [ ]:
# Load the three CSV files
users_df = pd.read_csv('data/users.csv')
events_df = pd.read_csv('data/events.csv')
items_df = pd.read_csv('data/items.csv')

print("Data loaded successfully!")
print(f"\nUsers shape: {users_df.shape}")
print(f"Events shape: {events_df.shape}")
print(f"Items shape: {items_df.shape}")

## 3. Explore Individual Datasets

In [ ]:
# Display first few rows of each dataset
print("=== USERS DATA ===")
print(users_df.head())
print(f"\nUsers data types:\n{users_df.dtypes}")
print(f"\nUsers info:")
print(users_df.info())

In [ ]:
print("=== EVENTS DATA ===")
print(events_df.head())
print(f"\nEvents data types:\n{events_df.dtypes}")
print(f"\nEvents info:")
print(events_df.info())

In [ ]:
print("=== ITEMS DATA ===")
print(items_df.head())
print(f"\nItems data types:\n{items_df.dtypes}")
print(f"\nItems info:")
print(items_df.info())

## 4. Data Cleaning and Preparation

In [ ]:
# Convert date columns to datetime
users_df['date'] = pd.to_datetime(users_df['date'])
events_df['date'] = pd.to_datetime(events_df['date'])

print("Date columns converted to datetime format")
print(f"\nUsers date range: {users_df['date'].min()} to {users_df['date'].max()}")
print(f"Events date range: {events_df['date'].min()} to {events_df['date'].max()}")

In [ ]:
# Check for missing values
print("Missing values in users_df:")
print(users_df.isnull().sum())
print("\nMissing values in events_df:")
print(events_df.isnull().sum())
print("\nMissing values in items_df:")
print(items_df.isnull().sum())

## 5. Join CSV Files Together

In [ ]:
# Filter events for purchases only (to calculate revenue)
purchases_df = events_df[events_df['type'] == 'purchase'].copy()

print(f"Total events: {len(events_df)}")
print(f"Purchase events: {len(purchases_df)}")
print(f"\nEvent types distribution:")
print(events_df['type'].value_counts())

In [ ]:
# Join purchases with items to get product details and prices
purchases_with_items = purchases_df.merge(
    items_df,
    left_on='item_id',
    right_on='id',
    how='left'
)

print("Purchases merged with items successfully!")
print(f"Merged dataset shape: {purchases_with_items.shape}")
print(f"\nFirst few rows:")
print(purchases_with_items.head())

In [ ]:
# Join with users data to get user information
full_dataset = purchases_with_items.merge(
    users_df,
    left_on='user_id',
    right_on='id',
    how='left',
    suffixes=('_event', '_user')
)

print("Full dataset created by joining all three CSV files!")
print(f"Full dataset shape: {full_dataset.shape}")
print(f"\nColumns in full dataset:")
print(full_dataset.columns.tolist())
print(f"\nFirst few rows:")
print(full_dataset.head())

## 6. Question 1: Which Categories Have the Highest Revenue?

In [ ]:
# Calculate revenue by category
revenue_by_category = full_dataset.groupby('category')['price_in_usd'].agg([
    ('total_revenue', 'sum'),
    ('num_purchases', 'count'),
    ('avg_price', 'mean')
]).sort_values('total_revenue', ascending=False)

print("Revenue by Category:")
print(revenue_by_category)
print(f"\nTop 3 Categories by Revenue:")
print(revenue_by_category.head(3))

In [ ]:
# Visualize revenue by category
plt.figure(figsize=(14, 8))
plt.subplot(2, 1, 1)
revenue_by_category['total_revenue'].plot(kind='bar', color='steelblue')
plt.title('Total Revenue by Product Category', fontsize=14, fontweight='bold')
plt.xlabel('Category', fontsize=12)
plt.ylabel('Total Revenue (USD)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

plt.subplot(2, 1, 2)
revenue_by_category['num_purchases'].plot(kind='bar', color='coral')
plt.title('Number of Purchases by Product Category', fontsize=14, fontweight='bold')
plt.xlabel('Category', fontsize=12)
plt.ylabel('Number of Purchases', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create a pie chart for revenue distribution
plt.figure(figsize=(12, 8))
top_n = 10
top_categories = revenue_by_category.head(top_n)
other_revenue = revenue_by_category.iloc[top_n:]['total_revenue'].sum()

# Add 'Other' category if there are more categories
if other_revenue > 0:
    plot_data = pd.concat([
        top_categories['total_revenue'],
        pd.Series({'Other': other_revenue})
    ])
else:
    plot_data = top_categories['total_revenue']

colors = plt.cm.Set3(range(len(plot_data)))
plt.pie(plot_data, labels=plot_data.index, autopct='%1.1f%%', colors=colors, startangle=90)
plt.title(f'Revenue Distribution by Category (Top {top_n} + Other)', fontsize=14, fontweight='bold')
plt.axis('equal')
plt.show()

## 7. Question 2: How Do Sales Change Over Time?

In [ ]:
# Extract date components for time-based analysis
full_dataset['year_month'] = full_dataset['date_event'].dt.to_period('M')
full_dataset['date_only'] = full_dataset['date_event'].dt.date
full_dataset['day_of_week'] = full_dataset['date_event'].dt.day_name()
full_dataset['hour'] = full_dataset['date_event'].dt.hour

print("Date components extracted successfully!")

In [ ]:
# Analyze sales by month
sales_by_month = full_dataset.groupby('year_month')['price_in_usd'].agg([
    ('total_revenue', 'sum'),
    ('num_transactions', 'count'),
    ('avg_transaction_value', 'mean')
]).reset_index()

sales_by_month['year_month'] = sales_by_month['year_month'].astype(str)

print("Sales by Month:")
print(sales_by_month)

In [ ]:
# Visualize sales over time (monthly)
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Total Revenue Over Time
axes[0].plot(sales_by_month['year_month'], sales_by_month['total_revenue'], 
             marker='o', linewidth=2, markersize=8, color='steelblue')
axes[0].set_title('Total Revenue Over Time (Monthly)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Month', fontsize=12)
axes[0].set_ylabel('Total Revenue (USD)', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Number of Transactions Over Time
axes[1].plot(sales_by_month['year_month'], sales_by_month['num_transactions'], 
             marker='s', linewidth=2, markersize=8, color='coral')
axes[1].set_title('Number of Transactions Over Time (Monthly)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Month', fontsize=12)
axes[1].set_ylabel('Number of Transactions', fontsize=12)
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

# Average Transaction Value Over Time
axes[2].plot(sales_by_month['year_month'], sales_by_month['avg_transaction_value'], 
             marker='^', linewidth=2, markersize=8, color='green')
axes[2].set_title('Average Transaction Value Over Time (Monthly)', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Month', fontsize=12)
axes[2].set_ylabel('Average Transaction Value (USD)', fontsize=12)
axes[2].tick_params(axis='x', rotation=45)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze sales by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sales_by_day = full_dataset.groupby('day_of_week')['price_in_usd'].agg([
    ('total_revenue', 'sum'),
    ('num_transactions', 'count')
]).reindex(day_order)

print("\nSales by Day of Week:")
print(sales_by_day)

In [ ]:
# Visualize sales by day of week
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].bar(sales_by_day.index, sales_by_day['total_revenue'], color='steelblue')
axes[0].set_title('Total Revenue by Day of Week', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Day of Week', fontsize=12)
axes[0].set_ylabel('Total Revenue (USD)', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

axes[1].bar(sales_by_day.index, sales_by_day['num_transactions'], color='coral')
axes[1].set_title('Number of Transactions by Day of Week', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Day of Week', fontsize=12)
axes[1].set_ylabel('Number of Transactions', fontsize=12)
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze sales by hour of day
sales_by_hour = full_dataset.groupby('hour')['price_in_usd'].agg([
    ('total_revenue', 'sum'),
    ('num_transactions', 'count')
]).reset_index()

print("\nSales by Hour of Day:")
print(sales_by_hour)

In [ ]:
# Visualize sales by hour
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

axes[0].plot(sales_by_hour['hour'], sales_by_hour['total_revenue'], 
             marker='o', linewidth=2, markersize=8, color='steelblue')
axes[0].set_title('Total Revenue by Hour of Day', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Hour of Day', fontsize=12)
axes[0].set_ylabel('Total Revenue (USD)', fontsize=12)
axes[0].set_xticks(range(0, 24))
axes[0].grid(True, alpha=0.3)

axes[1].plot(sales_by_hour['hour'], sales_by_hour['num_transactions'], 
             marker='s', linewidth=2, markersize=8, color='coral')
axes[1].set_title('Number of Transactions by Hour of Day', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Hour of Day', fontsize=12)
axes[1].set_ylabel('Number of Transactions', fontsize=12)
axes[1].set_xticks(range(0, 24))
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Additional Insights

In [ ]:
# Analyze sales by device type
sales_by_device = full_dataset.groupby('device')['price_in_usd'].agg([
    ('total_revenue', 'sum'),
    ('num_transactions', 'count'),
    ('avg_transaction_value', 'mean')
]).sort_values('total_revenue', ascending=False)

print("Sales by Device Type:")
print(sales_by_device)

In [ ]:
# Visualize sales by device
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].pie(sales_by_device['total_revenue'], labels=sales_by_device.index, 
            autopct='%1.1f%%', startangle=90, colors=['steelblue', 'coral', 'lightgreen'])
axes[0].set_title('Revenue Distribution by Device Type', fontsize=14, fontweight='bold')

axes[1].pie(sales_by_device['num_transactions'], labels=sales_by_device.index, 
            autopct='%1.1f%%', startangle=90, colors=['steelblue', 'coral', 'lightgreen'])
axes[1].set_title('Transaction Distribution by Device Type', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Top 10 products by revenue
top_products = full_dataset.groupby(['name', 'category'])['price_in_usd'].agg([
    ('total_revenue', 'sum'),
    ('units_sold', 'count')
]).sort_values('total_revenue', ascending=False).head(10)

print("\nTop 10 Products by Revenue:")
print(top_products)

In [ ]:
# Visualize top products
plt.figure(figsize=(12, 8))
top_products_plot = top_products.reset_index()
top_products_plot['product_label'] = top_products_plot['name'].str[:30]  # Truncate long names

plt.barh(range(len(top_products_plot)), top_products_plot['total_revenue'], color='steelblue')
plt.yticks(range(len(top_products_plot)), top_products_plot['product_label'])
plt.xlabel('Total Revenue (USD)', fontsize=12)
plt.title('Top 10 Products by Revenue', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()  # Highest revenue at top
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Summary of Findings

In [ ]:
print("="*80)
print("SUMMARY OF KEY FINDINGS")
print("="*80)

print("\n1. HIGHEST REVENUE CATEGORIES:")
print("-" * 40)
top_3_categories = revenue_by_category.head(3)
for idx, (category, row) in enumerate(top_3_categories.iterrows(), 1):
    print(f"   {idx}. {category}: ${row['total_revenue']:,.2f} ({row['num_purchases']:,} purchases)")

print("\n2. SALES TRENDS OVER TIME:")
print("-" * 40)
total_revenue = full_dataset['price_in_usd'].sum()
total_transactions = len(full_dataset)
avg_transaction = full_dataset['price_in_usd'].mean()
print(f"   Total Revenue: ${total_revenue:,.2f}")
print(f"   Total Transactions: {total_transactions:,}")
print(f"   Average Transaction Value: ${avg_transaction:.2f}")

print(f"\n   Peak Sales Month: {sales_by_month.loc[sales_by_month['total_revenue'].idxmax(), 'year_month']}")
print(f"   Peak Revenue: ${sales_by_month['total_revenue'].max():,.2f}")

print(f"\n   Best Day of Week: {sales_by_day['total_revenue'].idxmax()}")
print(f"   Best Hour of Day: {sales_by_hour.loc[sales_by_hour['total_revenue'].idxmax(), 'hour']}:00")

print("\n3. DEVICE TYPE PERFORMANCE:")
print("-" * 40)
for device, row in sales_by_device.iterrows():
    pct = (row['total_revenue'] / total_revenue) * 100
    print(f"   {device.capitalize()}: ${row['total_revenue']:,.2f} ({pct:.1f}% of total)")

print("\n" + "="*80)

## 10. Export Combined Dataset (Optional)

In [ ]:
# Save the combined dataset for future use
output_filename = 'data/combined_ecommerce_data.csv'
full_dataset.to_csv(output_filename, index=False)
print(f"Combined dataset saved to: {output_filename}")
print(f"Dataset contains {len(full_dataset)} records with {len(full_dataset.columns)} columns")